Import stuff

In [1]:
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime, timedelta
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import csv

In [2]:
ip_address = "YOUR IP"
client = MongoClient(f"{ip_address}", 27017)
db = client.fit3182_assignment_db
collection = db.climate_hotspot
query = collection.aggregate([{"$project" : {"_id" : 0, "date" : 1}}, {"$sort" : {"date" : -1}}, {"$limit" : 1}])
start_date = query.__next__()["date"]

In [3]:
start_date

datetime.datetime(2023, 1, 1, 0, 0)

Now we can load the data from climate_streaming.csv

In [4]:
climate_data = []
with open("climate_streaming.csv", "r") as climate_file:
    for climate in csv.DictReader(climate_file):
        ##change some data type
        climate["latitude"] = float(climate["latitude"])
        climate["longitude"] = float(climate["longitude"])
        climate["air_temperature_celcius"] = int(climate["air_temperature_celcius"])
        climate["relative_humidity"] = float(climate["relative_humidity"])
        climate["windspeed_knots"] = float(climate["windspeed_knots"])
        climate["max_wind_speed"] = float(climate["max_wind_speed"])
        climate["GHI_w/m2"] = int(climate["GHI_w/m2"])
        climate["precipitation_value"] = float(climate["precipitation "][:-1])
        climate["precipitation_type"] = climate["precipitation "][-1:]
        climate_data.append(climate)

Now to see a preview of the data to pass to the producer

In [5]:
data = {"datetime" : str(start_date.strftime("%d/%m/%Y %H:%M:%S")), "climate" : climate_data[random.randrange(0,len(climate_data))]}

In [6]:
data

{'datetime': '01/01/2023 00:00:00',
 'climate': {'latitude': -37.945,
  'longitude': 144.354,
  'air_temperature_celcius': 24,
  'relative_humidity': 55.6,
  'windspeed_knots': 5.9,
  'max_wind_speed': 13.0,
  'precipitation ': ' 0.00I',
  'GHI_w/m2': 196,
  'precipitation_value': 0.0,
  'precipitation_type': 'I'}}

Now to setup the kafka producer

In [7]:
host_ip = "YOUR IP"

def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [8]:
latest_date = datetime(start_date.year, start_date.month, start_date.day)
topic = 'Climate'

print('Publishing records..')
producer = connect_kafka_producer()

while True:
    data = {"datetime" : str(latest_date.strftime("%d/%m/%Y %H:%M:%S")), "climate" : climate_data[random.randrange(0,len(climate_data))]}
    publish_message(producer, topic, 'jsondata', data)
    latest_date = latest_date + timedelta(days=1)
    sleep(1)

Publishing records..
Message published successfully. Data: {'datetime': '01/01/2023 00:00:00', 'climate': {'latitude': -37.63, 'longitude': 149.232, 'air_temperature_celcius': 18, 'relative_humidity': 57.0, 'windspeed_knots': 7.4, 'max_wind_speed': 15.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 145, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '02/01/2023 00:00:00', 'climate': {'latitude': -37.013, 'longitude': 141.5355, 'air_temperature_celcius': 10, 'relative_humidity': 44.1, 'windspeed_knots': 7.3, 'max_wind_speed': 12.0, 'precipitation ': ' 0.39G', 'GHI_w/m2': 90, 'precipitation_value': 0.39, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '03/01/2023 00:00:00', 'climate': {'latitude': -36.4274, 'longitude': 142.1944, 'air_temperature_celcius': 22, 'relative_humidity': 46.9, 'windspeed_knots': 12.9, 'max_wind_speed': 19.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 193, 'precipitation_value': 0

Message published successfully. Data: {'datetime': '26/01/2023 00:00:00', 'climate': {'latitude': -37.396, 'longitude': 148.086, 'air_temperature_celcius': 15, 'relative_humidity': 41.6, 'windspeed_knots': 15.0, 'max_wind_speed': 28.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 138, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '27/01/2023 00:00:00', 'climate': {'latitude': -37.633, 'longitude': 149.264, 'air_temperature_celcius': 16, 'relative_humidity': 50.9, 'windspeed_knots': 12.9, 'max_wind_speed': 21.0, 'precipitation ': ' 0.03G', 'GHI_w/m2': 136, 'precipitation_value': 0.03, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '28/01/2023 00:00:00', 'climate': {'latitude': -37.189, 'longitude': 146.791, 'air_temperature_celcius': 21, 'relative_humidity': 61.1, 'windspeed_knots': 6.6, 'max_wind_speed': 11.1, 'precipitation ': ' 0.00I', 'GHI_w/m2': 163, 'precipitation_value': 0.0, 'precipitation_ty

Message published successfully. Data: {'datetime': '20/02/2023 00:00:00', 'climate': {'latitude': -37.3583, 'longitude': 143.0203, 'air_temperature_celcius': 13, 'relative_humidity': 43.3, 'windspeed_knots': 9.4, 'max_wind_speed': 14.0, 'precipitation ': ' 0.00G', 'GHI_w/m2': 118, 'precipitation_value': 0.0, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '21/02/2023 00:00:00', 'climate': {'latitude': -36.3096, 'longitude': 142.7722, 'air_temperature_celcius': 24, 'relative_humidity': 56.8, 'windspeed_knots': 5.7, 'max_wind_speed': 9.9, 'precipitation ': ' 0.00I', 'GHI_w/m2': 194, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '22/02/2023 00:00:00', 'climate': {'latitude': -37.401, 'longitude': 148.067, 'air_temperature_celcius': 13, 'relative_humidity': 42.5, 'windspeed_knots': 14.0, 'max_wind_speed': 19.0, 'precipitation ': ' 0.04G', 'GHI_w/m2': 118, 'precipitation_value': 0.04, 'precipitation_

Message published successfully. Data: {'datetime': '17/03/2023 00:00:00', 'climate': {'latitude': -37.288, 'longitude': 144.39, 'air_temperature_celcius': 12, 'relative_humidity': 49.9, 'windspeed_knots': 6.1, 'max_wind_speed': 13.0, 'precipitation ': ' 0.02G', 'GHI_w/m2': 103, 'precipitation_value': 0.02, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '18/03/2023 00:00:00', 'climate': {'latitude': -36.358, 'longitude': 143.113, 'air_temperature_celcius': 21, 'relative_humidity': 58.8, 'windspeed_knots': 8.8, 'max_wind_speed': 22.9, 'precipitation ': ' 0.08G', 'GHI_w/m2': 167, 'precipitation_value': 0.08, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '19/03/2023 00:00:00', 'climate': {'latitude': -37.38, 'longitude': 149.334, 'air_temperature_celcius': 16, 'relative_humidity': 48.2, 'windspeed_knots': 9.9, 'max_wind_speed': 18.1, 'precipitation ': ' 0.01G', 'GHI_w/m2': 139, 'precipitation_value': 0.01, 'precipitation_type

Message published successfully. Data: {'datetime': '11/04/2023 00:00:00', 'climate': {'latitude': -35.889, 'longitude': 145.614, 'air_temperature_celcius': 14, 'relative_humidity': 49.2, 'windspeed_knots': 10.5, 'max_wind_speed': 20.0, 'precipitation ': ' 0.12G', 'GHI_w/m2': 121, 'precipitation_value': 0.12, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '12/04/2023 00:00:00', 'climate': {'latitude': -35.779, 'longitude': 143.1057, 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'windspeed_knots': 9.0, 'max_wind_speed': 13.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 150, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '13/04/2023 00:00:00', 'climate': {'latitude': -36.6859, 'longitude': 146.8907, 'air_temperature_celcius': 16, 'relative_humidity': 51.5, 'windspeed_knots': 8.7, 'max_wind_speed': 15.0, 'precipitation ': ' 0.02G', 'GHI_w/m2': 135, 'precipitation_value': 0.02, 'precipitation

...

Message published successfully. Data: {'datetime': '31/05/2023 00:00:00', 'climate': {'latitude': -37.247, 'longitude': 141.278, 'air_temperature_celcius': 14, 'relative_humidity': 47.4, 'windspeed_knots': 7.1, 'max_wind_speed': 14.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 123, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '01/06/2023 00:00:00', 'climate': {'latitude': -37.437, 'longitude': 148.104, 'air_temperature_celcius': 12, 'relative_humidity': 40.7, 'windspeed_knots': 16.7, 'max_wind_speed': 21.0, 'precipitation ': ' 0.12G', 'GHI_w/m2': 111, 'precipitation_value': 0.12, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '02/06/2023 00:00:00', 'climate': {'latitude': -37.8387, 'longitude': 144.6301, 'air_temperature_celcius': 11, 'relative_humidity': 41.4, 'windspeed_knots': 7.7, 'max_wind_speed': 15.0, 'precipitation ': ' 0.03G', 'GHI_w/m2': 101, 'precipitation_value': 0.03, 'precipitation_

Message published successfully. Data: {'datetime': '25/06/2023 00:00:00', 'climate': {'latitude': -36.8264, 'longitude': 142.6138, 'air_temperature_celcius': 15, 'relative_humidity': 48.2, 'windspeed_knots': 14.7, 'max_wind_speed': 25.1, 'precipitation ': ' 0.43G', 'GHI_w/m2': 131, 'precipitation_value': 0.43, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '26/06/2023 00:00:00', 'climate': {'latitude': -37.479, 'longitude': 143.358, 'air_temperature_celcius': 23, 'relative_humidity': 60.6, 'windspeed_knots': 10.1, 'max_wind_speed': 26.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 180, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '27/06/2023 00:00:00', 'climate': {'latitude': -36.0279, 'longitude': 146.0244, 'air_temperature_celcius': 24, 'relative_humidity': 60.1, 'windspeed_knots': 8.0, 'max_wind_speed': 13.0, 'precipitation ': ' 0.12G', 'GHI_w/m2': 188, 'precipitation_value': 0.12, 'precipitati

Message published successfully. Data: {'datetime': '20/07/2023 00:00:00', 'climate': {'latitude': -37.978, 'longitude': 145.623, 'air_temperature_celcius': 21, 'relative_humidity': 59.5, 'windspeed_knots': 12.4, 'max_wind_speed': 21.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 166, 'precipitation_value': 0.0, 'precipitation_type': 'I'}}
Message published successfully. Data: {'datetime': '21/07/2023 00:00:00', 'climate': {'latitude': -36.778, 'longitude': 145.1635, 'air_temperature_celcius': 17, 'relative_humidity': 46.3, 'windspeed_knots': 11.7, 'max_wind_speed': 20.0, 'precipitation ': ' 0.02G', 'GHI_w/m2': 150, 'precipitation_value': 0.02, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '22/07/2023 00:00:00', 'climate': {'latitude': -35.89, 'longitude': 145.606, 'air_temperature_celcius': 15, 'relative_humidity': 45.6, 'windspeed_knots': 10.2, 'max_wind_speed': 15.0, 'precipitation ': ' 0.00G', 'GHI_w/m2': 133, 'precipitation_value': 0.0, 'precipitation_t

Message published successfully. Data: {'datetime': '14/08/2023 00:00:00', 'climate': {'latitude': -37.0684, 'longitude': 141.0437, 'air_temperature_celcius': 13, 'relative_humidity': 51.1, 'windspeed_knots': 6.0, 'max_wind_speed': 13.0, 'precipitation ': ' 0.12G', 'GHI_w/m2': 110, 'precipitation_value': 0.12, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '15/08/2023 00:00:00', 'climate': {'latitude': -37.34, 'longitude': 143.123, 'air_temperature_celcius': 19, 'relative_humidity': 52.8, 'windspeed_knots': 12.5, 'max_wind_speed': 21.0, 'precipitation ': ' 0.47G', 'GHI_w/m2': 159, 'precipitation_value': 0.47, 'precipitation_type': 'G'}}
Message published successfully. Data: {'datetime': '16/08/2023 00:00:00', 'climate': {'latitude': -36.3799, 'longitude': 143.7149, 'air_temperature_celcius': 12, 'relative_humidity': 43.8, 'windspeed_knots': 6.7, 'max_wind_speed': 14.0, 'precipitation ': ' 0.16G', 'GHI_w/m2': 108, 'precipitation_value': 0.16, 'precipitatio

KeyboardInterrupt: 